# Homework 8
This notebook provides some skeleton code to get you started on the homework. Add in your own code and markdown cells to answer the homework questions. If you want to submit the notebook as a PDF, make sure your code and markdowns are clear and concise to make grading easy for the TAs.
This notebook can be opened in Colab Open:
 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zlisto/social_media_analytics/blob/main/HW8.ipynb)


Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab. This will save the notebook in your Google Drive.





# Clones, installs, imports, and **GPU**

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [ ]:
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")

Cloning into 'social_media_analytics'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 435 (delta 96), reused 104 (delta 92), pack-reused 325
Receiving objects: 100% (435/435), 46.34 MiB | 10.94 MiB/s, done.
Resolving deltas: 100% (243/243), done.
Checking out files: 100% (55/55), done.


In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import umap
import scripts.TextAnalysis as ta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import codecs  #this let's us display tweets properly (emojis, etc.)
pd.set_option("display.max_colwidth", None)



```
```

# Task 1: Find User Communities to give Reflect an idea of the types of communities they can build.

## 1. (4 points) Load data

Load the tweets in the file `"data/oreo_sentiment.csv"` into a dataframe. Print out the head of this dataframe so we know you did it right.

In [ ]:
df = pd.read_csv("data/oreo_sentiment.csv")
ntweets = len(df)
print(f"dataframe has {ntweets} tweets")
df.head()

dataframe has 3000 tweets


,Unnamed: 0,created_at,screen_name,text,lang,retweet_count,reply_count,like_count,quote_count,id,author_id,conversation_id,in_reply_to_user_id,geo,sentiment
0,0,2022-01-28T22:03:44.000Z,Oreo,"@_JuiceJones What happened there? Let's fix this. Could you send us a DM with the package's UPC and expiration date, incl. the numbers/letters that follow? https://t.co/2mOFfNnQW0",en,1,0,1,0,1487184644600737794,126084292,1486849965955727364,8.274650e+07,None,2.102369
1,1,2022-01-28T21:36:06.000Z,Oreo,@ZeqoeR6 That's the best we have. 🤔,en,0,2,6,0,1487177690226597893,126084292,1486831768846188550,2.860886e+09,None,4.848118
2,2,2022-01-28T21:08:25.000Z,Oreo,@gagadaily the day all of our lives changed! truly iconic. https://t.co/zbJM37xbty,en,16,5,277,1,1487170722166452224,126084292,1487135476972761094,2.437448e+07,None,4.775955
3,3,2022-01-28T20:31:27.000Z,Oreo,@mswizzle44 They're available nationwide! We recommend checking larger retailers like Walmart in your area 🤞,en,0,2,1,0,1487161422895366146,126084292,1486714137996529666,4.646480e+09,None,4.485816
4,4,2022-01-28T19:25:47.000Z,Oreo,"@YaBoyShizo We'd like to get to the bottom of this! Could you send us a DM with the package's UPC and expiration date, incl. the numbers/letters that follow? https://t.co/2mOFfNnQW0",en,0,0,0,0,1487144893436895246,126084292,1486534320894455810,2.333453e+08,None,2.268613


## 2. (26 points) Cluster tweets

Clean the text with the `clean_tweet` function and calculate the tf-idf embedding.  Then calculate the UMAP embedding on the tf-idf embedding.  Finally, use k-means to cluster the UMAP embeddings into `k=4` clusters.  Make a count plot of the number of tweets in each cluster and a bar plot of the mean sentiment per cluster, with proper axis labels and title.  This question might look familiar to you (check a previous homework).

In [ ]:
df = pd.read_csv("data/oreo_sentiment.csv")
ntweets = len(df)

In [ ]:
df = pd.read_csv("data/oreo_sentiment.csv")
ntweets = len(df)
print(f"dataframe has {ntweets} tweets")
df['text_clean'] = df.text.apply(ta.clean_tweet)  #clean the tweets
df = df[df.text_clean.str.len() >0]  #remove cleaned tweets of lenght 0

nclean = len(df)

dataframe has 3000 tweets


In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, stop_words='english')
tfidf_embedding = tfidf_vectorizer.fit_transform(df.text_clean)

NameError: ignored

In [ ]:
umap_tfidf_embedding = umap.UMAP(n_components=2, metric='hellinger').fit_transform(tfidf_embedding)
umap_tfidf_embedding = stats.zscore(umap_tfidf_embedding,nan_policy='omit')
df['umap_tfidf_x'] = umap_tfidf_embedding[:,0]
df['umap_tfidf_y'] = umap_tfidf_embedding[:,1]


In [ ]:
kmeans_label = cluster.KMeans(n_clusters=n_clusters).fit_predict(umap_tfidf_embedding)
df['kmeans_label_tfidf_umap'] = [str(x) for x in kmeans_label]

plt.figure(figsize = (16,8))
ax = sns.countplot(data = df, x = 'kmeans_label_tfidf_umap')
plt.title(f"Tweets of Oreo by TF-IDF K-mean UMAP cluster",fontsize = 16)
plt.ylabel("Number of tweets",fontsize = 16)
plt.xlabel("Clusters",fontsize = 16)
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))
sns.barplot(data=df,x='kmeans_label_tfidf_umap', y="sentiment")
plt.ylabel("Sentiment", fontsize = 18)
plt.xlabel("Cluster", fontsize = 18)
plt.title(f"Sentiment per Cluster",fontsize=20)
plt.show()

## 3. (9 points) Plot mean sentiment vs mean retweet count per cluster

Make a scatterplot of the mean sentiment  vs the mean retweet count of each k-means cluster.  Set the marker size to 1000.   Make sure to add axes labels and a title to the figure.  

In [ ]:
for n_clusters in n_clusters:
  df['mean_sentiment']=df.sentiment.mean()
  df['mean_rtcount']=df.retweet_count.mean()

  fig = plt.figure(figsize = (12,8))
  ax = sns.scatterplot(data=df, x="sentiment", y="retweet_count")
  plt.xlabel("Sentiment", fontsize = 16)
  plt.ylabel("Retweet Count", fontsize = 16)
  plt.xticks(fontsize = 14)
  plt.yticks(fontsize = 14)

  plt.grid()
  plt.show()

In [ ]:
kmeans_label = cluster.KMeans(n_clusters=n_clusters).fit_predict(np.nan_to_num(umap_tfidf_embedding))
df['kmeans_label_tfidf_umap'] = [str(x) for x in kmeans_label]
s=1000
fig = plt.figure(figsize = (12,6))
sns.scatterplot(data = df, x= 'retweet_count', y = 'sentiment')
plt.xlim([3,4])
plt.xlabel("Screen name",fontsize = 16)
plt.ylabel("Mean sentiment",fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.title("Sentiment vs Screen Name",fontsize = 20)
plt.grid()

## 4. (8 points) Correlation of retweet count and sentiment

Calculate the Pearson correlation coefficient of the mean sentiment and mean retweet count of the clusters.  Print the coefficient and p-value up to three decimal points.  Is the correlation significant at a 1% level?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
my_rho = np.corrcoef(mean_sentiment, mean_rt)
print()

## 5. (10 points) T-test of cluster sentiment

Find the clusters with the lowest and highest mean sentiment and print their values.  Then use a t-test to determine if these mean sentiments are different at a 1% level.  Print your t-statistics, pvalues, and whether it is significant.

# Problem 2. (44 points) Tweet Keywords and Retweet Count

We will look at the retweet count as a function of the keyword in a tweet.

## 1. (4 points) Load data

Load the tweets in the file `"data/oreo_sentiment.csv"` into a dataframe. Print out the head of this dataframe so we know you did it right.

In [ ]:
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")

In [ ]:
from datetime import datetime, timedelta
import numpy as np
import sqlite3, sys, os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy import stats
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.multitest import multipletests

#import statsmodels.stats.multitest.multipletests as multipletests
#helper code
from scripts.api import *
import scripts.TextAnalysis as ta

pd.set_option("display.max_colwidth", None)

In [ ]:
fname = "data/oreo_sentiment.csv"
df_all = pd.read_csv(fname)
df_all.head()

## 2. (10 points) Plot retweet count vs. keyword

We will look at the keywords `"fordtrucks"`, `"thinsprotectionprogram"`,`"pokemon"`, and `"oreo"`.  For each keyword, make a separate bar plot of the mean retweet count of tweets with the keyword and without it.  Put labels on the axes and a title indicating which keyword you are plotting.  A `for` loop might be useful here.

In [ ]:
keywords = ['fordtrucks', 'thinsprotectionprogram','pokemon','oreo']

for keywords in keywords:
  fig = plt.figure(figsize = (12,6))
  df['has_keyword'] = df.text.str.contains(keywords, case = False)
  sns.barplot(data=df,x='has_keyword', y="retweet_count")
  plt.ylabel("Mean retweet count", fontsize = 18)
  plt.xlabel("Has keyword", fontsize = 18)
  plt.title(f"Tweets containing {keywords} mean retweet count",fontsize=20)
  plt.show()

## 3. (10 points) T-test of retweet count

Do a t-test for the mean retweet count of tweets with and without the keyword for each keyword.  Print your t-statistics and pvalues. A `for` loop might be useful here.

In [ ]:
keywords = ['fordtrucks', 'thinsprotectionprogram','pokemon','oreo']
for keyword in keywords:
  df[f'keyword_{keyword}'] = df.text.str.contains(keyword, case=False)

df.sample(n=2)

In [ ]:
keyword = ['fordtrucks', 'thinsprotectionprogram','pokemon','oreo']

X0 = df_u.retweet_count[df_u[f"{keyword}"]==False].dropna().tolist()
X1 = df_u.retweet_count[df_u[f"{keyword}"]==True].dropna().tolist()
n0 = len(X0)
n1 = len(X1)

mu0 = np.mean(X0)
mu1 = np.mean(X1)

print(f"Tweets of {screen_name}")
print(f"Contains {keyword}\tNumber of tweets\tMean retweet count")
print(f"False\t\t{n0}\t\t\t{mu0:.3f}")
print(f"True\t\t{n1}\t\t\t{mu1:.3f}")


## 4. (10 points) T-test of retweet count with Holm-Bonferroni Method

Apply the Holm-Bonferroni method to the calculated p-values.  Print the corrected pvalues and whether the mean retweet count difference is significant at a 1% level for each keyword.

## 5. (10 points) Oreo vs. Pokemon

Apply a t-test to compare the mean retweet count of tweets that contain `"pokemon"` to those that contain `"oreo"`.  Print the mean retweet count of each group of tweets, the t-statistic and p-value.  Is the difference significant at a 1% level?

# Problem 3. (24 points) Comparing Users

## 1. (4 points) Load data

Load the tweets in the database file `"data/HW4"` into a dataframe.  These are tweets for a few prominent Twitter users.

## 2. (10 points) Fisher exact test for keyword usage

Do a Fisher exact test to determine if users with screen names `"KingJames"` and `"Cristiano"` use the word `"nike"` with the same frequency.  Print the values in your contingency table (`a`,`b`,`c`,`d` defined in lecture).  Print the p-value and  whether the difference is significant at a 1% level.

## 3. (10 point) Follower engagement

Define follower engagement as the ratio of retweet count to follower count.  Do a t-test to determine if `"KingJames"` and `"Cristiano"` have the same follower engagement.  Print the mean follower engagement of each user, the t-statistic, p-value, and if the difference is significant at a 1% level.

The follower counts can be found in the file `"data/HW4"` in the `"users"` table.

# Problem 4. (20 points) Tweet Day

We will look at the effect the day of the week has on the retweet count for Lebron James.

## 1. (10 points) Plot retweet count vs day

From the tweets you loaded from the file `"data/HW4"`, get the tweets of the screen name `"KingJames"`.  Make a bar plot of the mean retweet count vs. day of the week.  Make sure to label axes and put the screen name in the title.

## 2. (10 points) T-test of Monday vs. Friday retweet count 

Apply a t-test to compare the mean retweet count of tweets posted on Monday vs. posted on Friday.  Print the mean retweet count of each group of tweets, the t-statistic and p-value.  Is the difference significant at a 1% level?